# 1. Учим conv сеть для классификации - выбить auc выше 0.95

In [1]:
max_words = 200
max_len = 40
num_classes = 2

# Training
epochs = 50
batch_size = 512
print_batch_n = 100

In [2]:
import pandas as pd

#df_train = pd.read_csv("data/train.csv")
#df_test = pd.read_csv("data/test.csv")
#df_val = pd.read_csv("data/val.csv")

df = pd.read_excel("отзывы за лето.xls")
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
df = df[df['Rating'] != 3]
df['target'] = df['Rating'] > 3
df['target'] = df['target'].astype(int)

### Предобработка

In [4]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date,target,text
0,5,It just works!,2017-08-14,1,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,1,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,1,удобно работать быстро


In [6]:
train_corpus = " ".join(df["text"])
#train_corpus = train_corpus.lower()

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [8]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [9]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [10]:
tokens_filtered_top[:20]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер',
 'быстро',
 'обновление',
 'пользоваться',
 'мочь',
 'банк',
 'антивирус',
 'устраивать',
 'сбербанк',
 'пароль',
 'карта']

In [11]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2,  random_state=42)
df_train, df_val  = train_test_split(df_train, test_size=0.2, random_state=42)

In [12]:
import numpy as np

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)


In [13]:
import keras

y_train = keras.utils.to_categorical(df_train["target"], num_classes)
y_val = keras.utils.to_categorical(df_val["target"], num_classes)
y_test = keras.utils.to_categorical(df_test["target"], num_classes)
#y_test = df_test["Rating"]

In [14]:
x_train.shape

(12638, 40)

In [15]:
max_len

40

In [16]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, 10])

# Keras model

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [18]:
model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=512, input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(100))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 512)           102400    
_________________________________________________________________
conv1d (Conv1D)              (None, 38, 128)           196736    
_________________________________________________________________
activation (Activation)      (None, 38, 128)           0         
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [20]:
early_stopping=EarlyStopping(monitor='val_auc',
                             patience=5,
                            mode="max",
                            restore_best_weights=True)  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping])

Epoch 1/50
25/25 [==============================] - 2s 64ms/step - loss: 0.4018 - auc: 0.9018 - val_loss: 0.2489 - val_auc: 0.9669
Epoch 2/50
25/25 [==============================] - 1s 47ms/step - loss: 0.2146 - auc: 0.9728 - val_loss: 0.1957 - val_auc: 0.9762
Epoch 3/50
25/25 [==============================] - 1s 47ms/step - loss: 0.1887 - auc: 0.9780 - val_loss: 0.1952 - val_auc: 0.9766
Epoch 4/50
25/25 [==============================] - 1s 47ms/step - loss: 0.1798 - auc: 0.9801 - val_loss: 0.1990 - val_auc: 0.9754
Epoch 5/50
25/25 [==============================] - 1s 46ms/step - loss: 0.1746 - auc: 0.9810 - val_loss: 0.1980 - val_auc: 0.9756
Epoch 6/50
25/25 [==============================] - 1s 46ms/step - loss: 0.1676 - auc: 0.9824 - val_loss: 0.1994 - val_auc: 0.9755
Epoch 7/50
25/25 [==============================] - 1s 53ms/step - loss: 0.1587 - auc: 0.9841 - val_loss: 0.2028 - val_auc: 0.9746
Epoch 8/50
25/25 [==============================] - 1s 47ms/step - loss: 0.1521 - a

In [21]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test auc:', score[1])


8/8 [==============================] - 0s 23ms/step - loss: 0.2040 - auc: 0.9746


Test loss: 0.20403659343719482
Test auc: 0.9745530486106873


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [22]:
!pip install gensim

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [23]:
from gensim.models import Word2Vec

In [24]:
df.head()

,Rating,Content,Date,target,text
0,5,It just works!,2017-08-14,1,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,1,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,1,удобно работать быстро


In [25]:
sentences = [[i for i in document.split()] for document in df['text'].tolist()]

In [26]:
model_w2v = Word2Vec(sentences=sentences, 
                     size=200, 
                     window=5,
                     min_count=2, 
                     sg=1, 
                     hs=0, 
                     negative=10, 
                     workers=32, 
                     seed=34)

model_w2v.train(sentences, total_examples=len(sentences), epochs=20)

(1320416, 1863800)

In [27]:
model_w2v.most_similar(positive=['минус'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('жирный', 0.6310212016105652),
 ('единственный', 0.6236435174942017),
 ('минусон', 0.6102328300476074),
 ('естьнедостаток', 0.5932835340499878),
 ('одинна', 0.587410032749176),
 ('интернетаня', 0.575370728969574),
 ('какнуть', 0.5721895098686218),
 ('тратиться', 0.5718035697937012),
 ('приложениепользоваться', 0.5657539367675781),
 ('проплатить', 0.5653463006019592)]

In [28]:
model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=200, input_length=max_len, 
                    weights=[model_w2v.trainables.syn1neg[:max_words]]))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))

model.add(GlobalMaxPool1D())

model.add(Dense(100))
model.add(Activation("relu"))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 200)           40000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 128)           76928     
_________________________________________________________________
activation_3 (Activation)    (None, 38, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12900     
_________________________________________________________________
activation_4 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['AUC'])

In [30]:
early_stopping=EarlyStopping(monitor='val_auc',
                             patience=10,
                            mode="max",
                            restore_best_weights=True)  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping])

Epoch 1/50
25/25 [==============================] - 1s 52ms/step - loss: 0.3780 - auc: 0.9123 - val_loss: 0.2880 - val_auc: 0.9503
Epoch 2/50
25/25 [==============================] - 1s 30ms/step - loss: 0.2611 - auc: 0.9595 - val_loss: 0.2385 - val_auc: 0.9660
Epoch 3/50
25/25 [==============================] - 1s 26ms/step - loss: 0.2172 - auc: 0.9718 - val_loss: 0.2139 - val_auc: 0.9724
Epoch 4/50
25/25 [==============================] - 1s 28ms/step - loss: 0.1920 - auc: 0.9775 - val_loss: 0.2033 - val_auc: 0.9745
Epoch 5/50
25/25 [==============================] - 1s 27ms/step - loss: 0.1791 - auc: 0.9801 - val_loss: 0.2068 - val_auc: 0.9735
Epoch 6/50
25/25 [==============================] - 1s 28ms/step - loss: 0.1713 - auc: 0.9818 - val_loss: 0.2046 - val_auc: 0.9745
Epoch 7/50
25/25 [==============================] - 1s 27ms/step - loss: 0.1635 - auc: 0.9834 - val_loss: 0.2086 - val_auc: 0.9739
Epoch 8/50
25/25 [==============================] - 1s 26ms/step - loss: 0.1504 - a

In [31]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test auc:', score[1])

8/8 [==============================] - 0s 15ms/step - loss: 0.2080 - auc: 0.9737


Test loss: 0.2080332487821579
Test auc: 0.9737425446510315


In [32]:
score

[0.2080332487821579, 0.9737425446510315]

0.9745 против 0.9737. Наверное можно сказать никак, хотя без w2v результат получился лучше.